In [1]:
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import folium
from folium.plugins import FastMarkerCluster
import numpy as np
import geopandas
import rtree
import descartes
import geopandas as gpd
from datetime import date, timedelta


In [2]:
number_drivers = 35

In [3]:

# Filepath
fp = "./Cambridge_Neighborhoods.shp"

# Read the data
pop = gpd.read_file(fp)
pop = pop[['NAME','geometry']]
pop=pop.to_crs(epsg=4326)


In [4]:
tomorrow = date.today()+ timedelta(days=1)
tomorrow = tomorrow.strftime("%m_%d")

In [5]:
filename ='./data/' + str(tomorrow) + "_deliveries.csv"
df_original = pd.read_csv(filename)

df = df_original.drop(columns = ['First Name', 'Last Name', 'Notes',
                                'Phone'])
df.head()

,ID,Street #,Street Name,Apt #,Senior,New,Gift Card
0,1,20,Chestnut Street,102,1.0,1.0,NaN
1,2,84,Jackson Street,3,1.0,1.0,NaN
2,3,101,Magazine St,3,1.0,1.0,NaN
3,4,2,Mount Auburn St,709,1.0,1.0,NaN
4,5,25,Park Ave,1st Floor,1.0,1.0,NaN


In [6]:
df.loc[df['Senior'].isnull(),'Senior']=0

In [7]:
#data cleaning
df['Street_clean']=df['Street Name']
for address in df['Street_clean']:
    try:
        df.loc[df.Street_clean==address,'Street_clean']=address.replace(".","")
        df.loc[df.Street_clean==address,'Street_clean']=address.replace(",","")
        df.loc[df.Street_clean==address,'Street_clean']=address.replace(";","")
        df.loc[df.Street_clean==address,'Street_clean']=address.replace("/","")
        df.loc[df.Street_clean==address,'Street_clean']=address.rstrip()
    except:
        print(address)

for address in df['Street_clean']:
    try:
        x=address.lower()
        
        if x[:2].isdigit():
            df.loc[df.Street_clean==address,'Street_clean']=x[x.find(" "):]

        if "-" in x:
            df.loc[df.Street_clean==address,'Street_clean']=x[:x.find("-")]
        if "(" in x:
            df.loc[df.Street_clean==address,'Street_clean']=x[:x.find("(")]
        if "calendar stret" in x:
            df.loc[df.Street_clean==address,'Street_clean']="Callender Street"
        if "alston street" in x:
            df.loc[df.Street_clean==address,'Street_clean']="Allston Street"
        if "auburn" in x:
            df.loc[df.Street_clean==address,'Street_clean']="Auburn Street"
        if "linwood pl" in x:
            df.loc[df.Street_clean==address,'Street_clean']="Broadway"
        if "mass ave" in x or "":
            df.loc[df.Street_clean==address,'Street_clean']="Massachusetts Avenue"
        if  "aveue" in x:
            df.loc[df.Street_clean==address,'Street_clean']=x[:x.find("aveue")] + "Avenue"
        if "stret" in x:
            df.loc[df.Street_clean==address,'Street_clean']=x[:x.find("Stret")] + "Street"
            df.loc[df.Street_clean==address,'Street_clean']=x[:x.find("stret")] + "Street"
        if  "cambridge park" in x:
            df.loc[df.Street_clean==address,'Street_clean']="Cambridgepark Drive"
        if "lansdown" in x or "lansdowne" in x:
            df.loc[df.Street_clean==address,'Street_clean']="Landsdowne St"
        if " apt " in x :
            df.loc[df.Street_clean==address,'Street_clean']=x[:x.find("apt")]
        if " bldg " in x :
            df.loc[df.Street_clean==address,'Street_clean']=x[:x.find("bldg")]
        if x[-4:]=='bldg':
            df.loc[df.Street_clean==address,'Street_clean']=x[:x.find("bldg")]
        if x[-3:]=='apt':
            df.loc[df.Street_clean==address,'Street_clean']=x[:x.find("apt")]
        if "frankln" in x or 'franlin' in x or 'frankin' in x or "frtanklin" in x:
            df.loc[df.Street_clean==address,'Street_clean']="Franklin Street"
        if "jackon" in x:
            df.loc[df.Street_clean==address,'Street_clean']="Jackson Pl"
        if "muesum" in x:
            df.loc[df.Street_clean==address,'Street_clean']="Museum Way"
        if "newtown" in x or "newton" in x:
            df.loc[df.Street_clean==address,'Street_clean']="Main Street"
            df.loc[df.Street_clean==address,'Street #']="637"
        if "eerie" in x:
            df.loc[df.Street_clean==address,'Street_clean']="Erie Street"
        if "8th" in x:
            df.loc[df.Street_clean==address,'Street_clean']="Eighth Street"
        if "cliffton" in x:
            df.loc[df.Street_clean==address,'Street_clean']="Clifton Place"
    except:
        print(address)

In [8]:
df['ADDRESS']=""
for address in df['ADDRESS']:
    if address == "":
        df['ADDRESS'] = df['Street #'].astype(str) + ' ' + \
                        df['Street_clean'] + ',' + \
                        "Cambridge" + ',' + \
                        "Massachusetts" + ',' + ' USA'   


In [9]:
good_points = pd.read_csv("point_table.csv")

In [10]:
len(good_points)

942

In [11]:
point_table= df.merge(good_points,on = ['ADDRESS'],how='left')
bad_table = point_table.loc[point_table['point_location'].isnull()]
good_table = point_table.dropna(subset=['point_location'], how='all')


In [13]:
len(good_table), len(bad_table), len(df)

(325, 25, 350)

In [15]:
def geocode(df):
    from geopy.extra.rate_limiter import RateLimiter
    locator = Nominatim(user_agent="myGeocoder")
    geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
    df['location'] = df['ADDRESS'].apply(geocode)
    df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
    null_df = df.loc[df['point'].isnull()]
    null_df[['latitude', 'longitude', 'point_location']] = None
    df=df.dropna(subset=['point'], how='all')
    
    print(len(df), len(df.index), len(df['point']))
    try:
        df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)
        df['point_location'] = df["longitude"].astype(str) + "," + df["latitude"].astype(str)
        df = df.drop(columns = ['point','altitude'])
    except:
        df[['latitude', 'longitude','point_location']]=None,None,None
    null_df = null_df.drop(columns = ['point'])

    return df, null_df

In [16]:
geocoded_table, null_table = geocode(bad_table)

24 24 24


/Users/ericenglin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/ericenglin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/ericenglin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [18]:
df_new = pd.concat([good_table,geocoded_table])

In [19]:
locations = df_new[['latitude', 'longitude']]
locationlist = locations.values.tolist()

In [20]:
df_new = df_new.reset_index()


In [21]:
map1 = folium.Map(
    location=[42.362750, -71.101182],
    tiles='cartodbpositron',
    zoom_start=12,
)

df_new.apply(lambda row:folium.Marker(
    location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=('Address: '+str(df_new['ADDRESS'][point])),
                 icon=folium.Icon(color='blue')).add_to(map1)

map1


In [22]:
gdf = geopandas.GeoDataFrame(
    df_new, geometry=geopandas.points_from_xy(df_new.longitude.astype(float), df_new.latitude.astype(float)))

df_new = gpd.sjoin(gdf, pop, how="left", op="within")

/Users/ericenglin/opt/anaconda3/lib/python3.7/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!(None != epsg:4326)
  "(%s != %s)" % (left_df.crs, right_df.crs)


In [23]:
color_dict = {-1:'darkgreen',0:'darkpurple',1:'green',2:'purple',3:'red',4:'gray',5:'lightblue',
             6:'beige',7:'cadetblue',8:'black',9:'pink',10:'darkred',11:'lightred',
             12:'darkblue',13:'lightgreen',13:'white',14:'darkgreen',15:'darkpurple',16:'green',
              17:'purple',18:'red',19:'gray',20:'lightblue',
             21:'beige',22:'cadetblue',23:'black',24:'pink',25:'darkred',26:'lightred',
             27:'darkblue',28:'lightgreen',29:'white',30:'gray',31:'cadetblue',32:'orange',33:'green',
              34:'white',35:'darkgreen'}

In [24]:
df_new.loc[df_new['NAME'].isnull()]

,index,ID,Street #,Street Name,Apt #,Senior,New,Gift Card,Street_clean,ADDRESS,location,latitude,longitude,point_location,geometry,index_right,NAME
338,125,126,11,Ware Street,5,1.0,NaN,NaN,Ware Street,"11 Ware Street,Cambridge,Massachusetts, USA","(11, Ware Street, West Somerville, Somerville,...",42.405177,-71.1268,"-71.12684486852682,42.4051771",POINT (-71.12684 42.40518),NaN,NaN


In [25]:
not_cambridge_df = df_new.loc[df_new['NAME'].isnull()]
null_table = pd.concat([null_table,not_cambridge_df])
df_new = df_new.dropna(subset=['NAME'], how='all')

len(null_table), len(df_new)

/Users/ericenglin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


(2, 348)

In [26]:
def neighborhood_clean(df):
    neighborhood_list_temp = []
    
    neighborhood_list = ['The Port','Neighborhood Nine','Area 2/MIT',
                    'Cambridgeport','Riverside','Mid-Cambridge',
                    'Wellington-Harrington','East Cambridge','Agassiz','Cambridge Highlands',
                    'Strawberry Hill','West Cambridge','North Cambridge']
    
    for x in df_new['NAME']:
        z=0
        for y in neighborhood_list:
            if y in str(x):
                if y in ["Cambridge Highlands","West Cambridge", "Strawberry Hill"]:
                    neighborhood_list_temp.append('North Cambridge')
                elif y in ["Agassiz","Neighborhood Nine","Riverside"]:
                    neighborhood_list_temp.append('Mid-Cambridge')
                elif y == "Area 2/MIT":
                    neighborhood_list_temp.append('Cambridgeport')
                elif y in ["Wellington-Harrington","The Port"]:
                    neighborhood_list_temp.append('East Cambridge')
                else: 
                    neighborhood_list_temp.append(y)
                z=1
                break
        if z==0:
            neighborhood_list_temp.append('Problem Location')
        
    return neighborhood_list_temp

In [27]:
df_new['neighborhood']=neighborhood_clean(df_new)
df_new['neighborhood'].value_counts()

East Cambridge     116
Cambridgeport       98
North Cambridge     83
Mid-Cambridge       51
Name: neighborhood, dtype: int64

In [28]:
neighborhood_color_dict = {'Cambridgeport':'darkgreen','East Cambridge':'darkpurple',
                           'North Cambridge':'purple','Mid-Cambridge':'black',
                           'Problem Location':'red'}

In [29]:
df_new['neighborhood_color']=df_new['neighborhood'].map(neighborhood_color_dict)


In [30]:
neighborhood_map_name = 'DeliveryNeighborhoods_' + str(tomorrow) + '.html'

In [31]:
df_new = df_new.reset_index()
locations = df_new[['latitude', 'longitude']]
locationlist = locations.values.tolist()

In [32]:
map = folium.Map(location=[42.379750, -71.101182], zoom_start=13)
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=('Driver Group: '+str(df_new['neighborhood'][point])),
                 icon=folium.Icon(color=df_new["neighborhood_color"][point])).add_to(map)
    
map.save(neighborhood_map_name)
map

In [33]:
# Get one hot encoding of columns B
one_hot = pd.get_dummies(df_new['neighborhood'])
# Drop column B as it is now encoded
df_new = df_new.drop('neighborhood',axis = 1)
# Join the encoded df
df_new = df_new.join(one_hot)

In [34]:
def run_kmeans(df, n_clusters, min_size, max_size):
    from k_means_constrained import KMeansConstrained
    df_temp = df[['latitude', 'longitude']]
    # Convert DataFrame to matrix
    mat = df_temp.values
    # Using sklearn
    km = KMeansConstrained(
         n_clusters=n_clusters,
         size_min=min_size,
         size_max=max_size,
         random_state=0
    ).fit(mat)
    # Get cluster assignment labels
    labels = km.labels_
    return labels

In [35]:
df_new = df_new.sort_values(['Senior'])
df_seniors = df_new.loc[df_new['Senior']==1]
df_nonseniors = df_new.loc[df_new['Senior']!=1]
senior_clusters = int(round(len(df_seniors)/10,0))
nonsenior_clusters = number_drivers - senior_clusters

In [36]:
len(df_seniors)

199

In [37]:
df_senior_labels = run_kmeans(df_seniors, int(senior_clusters),9,12)
df_nonsenior_labels = run_kmeans(df_nonseniors, int(nonsenior_clusters),9,12)
df_nonsenior_labels = df_nonsenior_labels + senior_clusters

all_labels = np.concatenate([df_nonsenior_labels, df_senior_labels])
df_new['kmeans_cluster'] = all_labels
df_new['kmeans_cluster'].value_counts()

/Users/ericenglin/opt/anaconda3/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


32    12
9     12
2     12
33    12
8     12
4     12
20    12
23    11
3     11
7     10
16    10
19    10
0     10
5     10
26    10
27    10
29    10
6      9
12     9
1      9
10     9
11     9
34     9
13     9
14     9
15     9
18     9
21     9
22     9
24     9
25     9
28     9
30     9
31     9
17     9
Name: kmeans_cluster, dtype: int64

In [38]:
df_seniors['kmeans_cluster'] = df_senior_labels

df_seniors = df_seniors.drop(columns = ['level_0'])
df_seniors = df_seniors.reset_index()

senior_locations = df_seniors[['latitude', 'longitude']]
seniorlocationlist = senior_locations.values.tolist()

/Users/ericenglin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [39]:
df_seniors['kmeans_color']=df_seniors['kmeans_cluster'].map(color_dict)

In [40]:
map = folium.Map(location=[42.379750, -71.101182], zoom_start=13)
for point in range(0, len(seniorlocationlist)):
    folium.Marker(seniorlocationlist[point], popup=('Driver Group: '+str(df_seniors['kmeans_cluster'][point])),
                 icon=folium.Icon(color=df_seniors["kmeans_color"][point])).add_to(map)
    
#map.save(delivery_map_name)
map

In [41]:
df_new['kmeans_color']=df_new['kmeans_cluster'].map(color_dict)

In [42]:
delivery_map_name = 'DeliveryMaps_' + str(tomorrow) + '.html'

In [43]:
map = folium.Map(location=[42.379750, -71.101182], zoom_start=13)
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=('Driver Group: '+str(df_new['kmeans_cluster'][point])),
                 icon=folium.Icon(color=df_new["kmeans_color"][point])).add_to(map)
    
map.save(delivery_map_name)
map

In [44]:
point_table.columns

Index(['ID', 'Street #', 'Street Name', 'Apt #', 'Senior', 'New', 'Gift Card',
       'Street_clean', 'ADDRESS', 'location', 'latitude', 'longitude',
       'point_location'],
      dtype='object')

In [45]:
#update points in point table
df_newpoints = df_new[['ADDRESS','location',  
                          'latitude', 'longitude', 
                          'point_location']]


new_point_table = pd.concat([df_newpoints, good_points])
new_point_table = new_point_table.drop_duplicates(subset=['ADDRESS']).dropna(subset=['ADDRESS'], 
                                                                             how='all').sort_values(by = ['ADDRESS'])
new_point_table.to_csv('point_table.csv',index=False)

In [46]:
df_new['Driver_ID']=df_new['kmeans_cluster']
null_table['Driver_ID']=99


In [47]:
driver_table = pd.concat([null_table,df_new])

/Users/ericenglin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [48]:
driver_table_final = driver_table.merge(df_original[['ID','First Name', 'Last Name', 'Notes',
                                'Phone']], on = 'ID')

In [49]:
driver_table_final=driver_table_final[['ID','First Name', 'Last Name',
                                        'Street #','Street Name','Apt #',
                                        'Driver_ID', 'New','Senior','Gift Card','Phone',
                                        'Notes','ADDRESS','location','latitude','longitude']].sort_values(by = ['Driver_ID', 'Street Name', 
                                                       'Street #', 'Apt #'],ascending=False)


In [50]:
file_name_final = "./delivery routes/Driver_Table_" + str(tomorrow) + ".xlsx"

In [51]:
driver_table_final.to_excel(file_name_final, index=False)